
Step 1. [Download history price and clean data](/notebooks/forex/hawkeye/1_download.ipynb)

Step 2. [Use monte carlo simulate each stock's movement for 10000 times](/notebooks/forex/hawkeye/2_process.ipynb)

**Step 3. Apply some heuristic rules to give stock recommendation**

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import sys

sys.path.append('/opt/hawkeye')
from asx import *

this_week = 20190614
last_week = 20190607
path = f'/opt/hawkeye/data/{this_week}/result.csv'
last_path = f'/opt/hawkeye/data/{last_week}/result.csv'

def week_return_avg(df):
    return round(df['week_return'].dropna().sum()/len(df['week_return'].dropna()),4)

df = pd.read_csv(path,
                 usecols=[
                     'code', 'last_date', 'start price', 'sim_mean',
                     'sim_diff', 'VaR 99%', 'VaR 99% Percent', 'volume_mean',
                     'return_mean', 'return_sigma'
                 ],
                 index_col='code')
last_df = pd.read_csv(last_path,
                      usecols=[
                          'code', 'last_date', 'start price', 'sim_mean',
                          'sim_diff', 'VaR 99%', 'VaR 99% Percent',
                          'volume_mean', 'return_mean', 'return_sigma'
                      ],
                      index_col='code')

print(f'Total stock number = {len(df)}')

# remove bottom volume and bottom price
volume_threshold = df['volume_mean'].quantile(0.1)
df = df[(df['volume_mean'] > volume_threshold) & (df['start price'] > 0.05)]

# calculate return & risk rank
df['return'] = round(df['sim_diff'] / df['start price'] * 100, 3)
df['return_rank'] = round(df['return'].rank(pct=True) * 100, 3)
df['risk_rank'] = round(df['VaR 99% Percent'].rank(pct=True) * 100, 3)
df['volume_rank'] = round(df['volume_mean'].rank(pct=True) * 100, 3)

# last week performance
df['last_return'] = round(last_df['sim_diff'] / last_df['start price'] * 100,3)
df['return_increase'] = df['return'] - df['last_return']
df['return_increase_rank'] = round(df['return_increase'].rank(pct=True) * 100, 3)
df['week_return'] = round(
    (df['start price'] - last_df['start price']) / df['start price'] * 100, 3)

# drop used columns
df.drop(columns=['sim_diff', 'sim_mean', 'VaR 99%','VaR 99% Percent', 'volume_mean', 'return_increase'], inplace=True)

print(f'Total stock number after bottom volume & price removed = {len(df)}')
print('Price 70%% pricetile = %s' % df['start price'].quantile(0.7))
print('Sim return 90%% percentile = %s' % df['return'].quantile(0.9))

df.sort_values(by='return', ascending=False).head(20)

Total stock number = 1616
Total stock number after bottom volume & price removed = 998
Price 70% pricetile = 2.018
Sim return 90% percentile = 1.0726000000000009


,last_date,start price,return_mean,return_sigma,return,return_rank,risk_rank,volume_rank,last_return,return_increase_rank,week_return
code,,,,,,,,,,,
FTC,2019-06-14,0.155,0.025928,0.190769,5.079,100.000,99.800,73.547,4.869,90.430,6.452
UWL,2019-06-14,1.310,0.024609,0.075059,4.815,99.900,82.465,79.760,NaN,NaN,NaN
LTR,2019-06-14,0.115,0.019955,0.096789,3.860,99.800,96.894,99.499,3.855,56.022,8.696
FEX,2019-06-14,0.110,0.019293,0.089630,3.848,99.699,94.790,98.898,4.226,4.785,-4.545
8CO,2019-06-14,0.145,0.019064,0.076112,3.803,99.599,86.072,50.401,3.857,37.634,-10.345
NET,2019-06-14,0.110,0.019104,0.086727,3.732,99.499,92.585,91.082,2.954,99.570,12.727
ANO,2019-06-14,5.920,0.018382,0.068696,3.653,99.399,80.561,28.257,3.785,20.269,-9.459
TNY,2019-06-14,1.190,0.018666,0.109975,3.639,99.299,99.198,31.363,NaN,NaN,NaN
ISX,2019-06-14,0.635,0.016686,0.056640,3.315,99.198,68.036,87.475,3.232,76.237,-2.362


In [6]:
# high mean return 
high_return=df.sort_values(by='return', ascending=False).head(10)
print('High return last week performance avg. = %s%%'% week_return_avg(high_return))
high_return

High return last week performance avg. = -0.3353%


,last_date,start price,return_mean,return_sigma,return,return_rank,risk_rank,volume_rank,last_return,return_increase_rank,week_return
code,,,,,,,,,,,
FTC,2019-06-14,0.155,0.025928,0.190769,5.079,100.000,99.800,73.547,4.869,90.430,6.452
UWL,2019-06-14,1.310,0.024609,0.075059,4.815,99.900,82.465,79.760,NaN,NaN,NaN
LTR,2019-06-14,0.115,0.019955,0.096789,3.860,99.800,96.894,99.499,3.855,56.022,8.696
FEX,2019-06-14,0.110,0.019293,0.089630,3.848,99.699,94.790,98.898,4.226,4.785,-4.545
8CO,2019-06-14,0.145,0.019064,0.076112,3.803,99.599,86.072,50.401,3.857,37.634,-10.345
NET,2019-06-14,0.110,0.019104,0.086727,3.732,99.499,92.585,91.082,2.954,99.570,12.727
ANO,2019-06-14,5.920,0.018382,0.068696,3.653,99.399,80.561,28.257,3.785,20.269,-9.459
TNY,2019-06-14,1.190,0.018666,0.109975,3.639,99.299,99.198,31.363,NaN,NaN,NaN
ISX,2019-06-14,0.635,0.016686,0.056640,3.315,99.198,68.036,87.475,3.232,76.237,-2.362


In [7]:
# high price

high_price=df[df['start price']>1].sort_values(by='return', ascending=False).head(10)
print('High price last week performance avg. = %s%%'% week_return_avg(high_price))
high_price

High price last week performance avg. = 5.6018%


,last_date,start price,return_mean,return_sigma,return,return_rank,risk_rank,volume_rank,last_return,return_increase_rank,week_return
code,,,,,,,,,,,
UWL,2019-06-14,1.310,0.024609,0.075059,4.815,99.900,82.465,79.760,NaN,NaN,NaN
ANO,2019-06-14,5.920,0.018382,0.068696,3.653,99.399,80.561,28.257,3.785,20.269,-9.459
TNY,2019-06-14,1.190,0.018666,0.109975,3.639,99.299,99.198,31.363,NaN,NaN,NaN
DUB,2019-06-14,1.420,0.014615,0.059723,2.810,98.798,76.253,70.741,2.318,98.495,21.127
CIA,2019-06-14,3.240,0.011548,0.039920,2.274,97.896,48.196,55.812,1.960,95.591,9.568
BUB,2019-06-14,1.075,0.010646,0.056375,2.108,97.194,72.946,96.192,2.099,56.882,-0.465
Z1P,2019-06-14,2.950,0.010513,0.046225,2.045,97.094,57.315,89.379,2.310,8.172,-2.712
5GN,2019-06-14,1.420,0.010393,0.046456,1.987,96.894,59.619,61.323,1.720,93.441,17.606
CRD,2019-06-14,1.740,0.009947,0.045742,1.960,96.794,57.715,52.405,1.866,78.065,1.149


In [8]:
# highest price

highest_price=df[df['return']>0.8].sort_values(by='start price', ascending=False).head(10)
print('Highest price last week performance avg. = %s%%'% week_return_avg(highest_price))
highest_price

Highest price last week performance avg. = 3.8538%


,last_date,start price,return_mean,return_sigma,return,return_rank,risk_rank,volume_rank,last_return,return_increase_rank,week_return
code,,,,,,,,,,,
MFG,2019-06-14,47.60,0.005717,0.016877,1.099,90.481,9.319,67.435,1.083,59.731,3.718
CUV,2019-06-14,34.10,0.005243,0.036112,1.036,89.579,49.699,35.170,0.771,93.226,7.067
APX,2019-06-14,25.82,0.005689,0.033277,1.100,90.581,45.190,73.647,1.245,18.441,-3.447
JIN,2019-06-14,18.75,0.009431,0.029942,1.872,96.593,30.661,55.611,1.692,88.172,8.000
IEL,2019-06-14,18.03,0.005005,0.026398,0.960,88.577,31.764,71.042,0.964,53.065,7.321
BRG,2019-06-14,16.97,0.005061,0.023582,0.992,88.978,25.451,55.411,0.933,71.183,1.414
CHC,2019-06-14,11.52,0.004294,0.011989,0.826,85.271,2.655,83.467,0.743,76.398,5.035
LOV,2019-06-14,11.28,0.005302,0.032048,1.069,89.980,44.188,52.806,1.223,16.774,-0.355
APE,2019-06-14,9.90,0.005588,0.022739,1.056,89.880,21.643,14.529,1.052,55.806,-1.010


In [9]:
# low risk
low_risk = df.sort_values(by='risk_rank', ascending=True).head(20).sort_values(by='return', ascending=False)
print('Low risk last week performance avg. = %s%%'% week_return_avg(low_risk))
low_risk

Low risk last week performance avg. = 1.5579%


,last_date,start price,return_mean,return_sigma,return,return_rank,risk_rank,volume_rank,last_return,return_increase_rank,week_return
code,,,,,,,,,,,
ASX,2019-06-14,81.000,0.002568,0.009643,0.496,73.597,1.904,57.214,0.495,54.731,2.111
TCL,2019-06-14,14.510,0.002049,0.008477,0.406,67.635,1.503,95.491,0.380,62.419,1.723
SHL,2019-06-14,27.500,0.002129,0.008969,0.405,67.435,1.603,76.854,0.411,52.473,3.709
CMA,2019-06-14,2.830,0.001996,0.007329,0.385,66.132,0.601,67.034,0.337,69.247,3.180
AST,2019-06-14,1.945,0.001964,0.008382,0.384,65.982,1.202,94.589,0.342,67.312,4.627
CMW,2019-06-14,1.200,0.001892,0.008086,0.371,65.431,0.802,93.287,0.356,59.462,0.833
AMC,2019-06-14,15.650,0.001951,0.009398,0.363,64.880,2.004,93.788,0.402,42.151,-0.767
GOZ,2019-06-14,4.320,0.001708,0.007992,0.331,62.876,1.002,68.136,0.301,63.763,0.694
CIP,2019-06-14,3.210,0.001605,0.008074,0.316,61.774,1.303,40.982,0.307,57.258,0.623


In [10]:
# high volume
high_volume = df.sort_values(by='volume_rank', ascending=False).head(20).sort_values(by='return', ascending=False)
print('Highest volume last week performance avg. = %s%%'% week_return_avg(high_volume))
high_volume

Highest volume last week performance avg. = 2.1966%


,last_date,start price,return_mean,return_sigma,return,return_rank,risk_rank,volume_rank,last_return,return_increase_rank,week_return
code,,,,,,,,,,,
LTR,2019-06-14,0.115,0.019955,0.096789,3.860,99.800,96.894,99.499,3.855,56.022,8.696
FEX,2019-06-14,0.110,0.019293,0.089630,3.848,99.699,94.790,98.898,4.226,4.785,-4.545
AVH,2019-06-14,0.360,0.013883,0.072828,2.689,98.597,89.178,98.497,2.774,29.839,5.556
FMG,2019-06-14,8.800,0.007762,0.029624,1.586,94.990,33.166,99.900,1.370,90.645,10.795
CLA,2019-06-14,0.064,0.007115,0.101932,1.109,90.681,99.299,98.196,0.357,99.462,32.813
MGR,2019-06-14,3.210,0.003611,0.011091,0.697,82.114,2.104,99.599,0.621,74.946,2.492
TLS,2019-06-14,3.870,0.003052,0.012511,0.621,79.559,5.010,100.000,0.521,78.925,3.876
NEC,2019-06-14,2.000,0.002747,0.023184,0.572,76.854,27.154,98.798,0.592,48.011,1.750
MPL,2019-06-14,3.380,0.002771,0.017488,0.568,76.703,15.431,98.597,0.552,59.570,0.888


In [11]:
# high return increase in last week
high_return_increase = df.sort_values(by='return_increase_rank',
                                      ascending=False).head(10)
print('Highest return increase last week performance avg. = %s%%'% week_return_avg(high_return_increase))
high_return_increase

Highest return increase last week performance avg. = 21.7534%


,last_date,start price,return_mean,return_sigma,return,return_rank,risk_rank,volume_rank,last_return,return_increase_rank,week_return
code,,,,,,,,,,,
CUP,2019-06-14,0.750,0.004316,0.045827,0.815,84.770,65.832,14.128,-0.286,100.000,40.000
BKY,2019-06-14,0.385,0.000805,0.064948,0.130,48.547,90.481,45.591,-0.885,99.892,24.675
FZO,2019-06-14,0.135,0.000185,0.072066,0.143,49.800,93.687,50.802,-0.757,99.785,14.815
FMS,2019-06-14,0.052,0.006501,0.068578,1.288,92.485,88.878,46.593,0.434,99.677,28.846
NET,2019-06-14,0.110,0.019104,0.086727,3.732,99.499,92.585,91.082,2.954,99.570,12.727
CLA,2019-06-14,0.064,0.007115,0.101932,1.109,90.681,99.299,98.196,0.357,99.462,32.813
GSM,2019-06-14,0.080,-0.005263,0.046054,-1.038,3.607,77.555,1.603,-1.778,99.355,25.000
BSX,2019-06-14,0.096,0.001123,0.055311,0.230,56.814,82.966,37.174,-0.385,99.247,19.792
CGA,2019-06-14,0.325,0.004533,0.059873,0.952,88.327,83.768,0.100,0.353,99.140,3.077


In [12]:
# ASX 20
asx_20_list = get_asx_20_list()
asx_20=df[df.index.isin(asx_20_list)].sort_values(by='return', ascending=False)
print('ASX 20 last week performance avg. = %s%%'% week_return_avg(asx_20))
asx_20

ASX 20 last week performance avg. = 1.3436%


,last_date,start price,return_mean,return_sigma,return,return_rank,risk_rank,volume_rank,last_return,return_increase_rank,week_return
code,,,,,,,,,,,
TLS,2019-06-14,3.87,0.003052,0.012511,0.621,79.559,5.010,100.000,0.521,78.925,3.876
RIO,2019-06-14,105.34,0.003105,0.015542,0.583,77.555,11.323,84.469,0.479,79.624,6.968
BHP,2019-06-14,40.30,0.002273,0.012704,0.460,71.293,7.014,97.395,0.323,83.656,6.104
BXB,2019-06-14,12.97,0.002152,0.010432,0.420,68.788,3.607,93.487,0.405,59.247,1.850
TCL,2019-06-14,14.51,0.002049,0.008477,0.406,67.635,1.503,95.491,0.380,62.419,1.723
AMC,2019-06-14,15.65,0.001951,0.009398,0.363,64.880,2.004,93.788,0.402,42.151,-0.767
WES,2019-06-14,35.66,0.001249,0.013623,0.252,58.517,11.222,88.677,0.368,24.140,-5.580
ANZ,2019-06-14,28.22,0.001168,0.014696,0.248,58.216,12.926,96.293,0.297,39.516,-0.532
CBA,2019-06-14,79.77,0.001248,0.011746,0.241,57.615,5.812,90.381,0.249,51.613,-0.288


In [13]:
# highest return in ASX 200
asx_200_list = get_asx_200_list()
asx_200=df[df.index.isin(asx_200_list)].sort_values(by='return', ascending=False).head(20)
print('Top 20 of ASX 200 last week performance avg. = %s%%'% week_return_avg(asx_200))
asx_200

Top 20 of ASX 200 last week performance avg. = 3.0694%


,last_date,start price,return_mean,return_sigma,return,return_rank,risk_rank,volume_rank,last_return,return_increase_rank,week_return
code,,,,,,,,,,,
FMG,2019-06-14,8.80,0.007762,0.029624,1.586,94.990,33.166,99.900,1.370,90.645,10.795
LYC,2019-06-14,2.77,0.006721,0.050607,1.332,93.186,68.437,97.595,1.186,85.376,0.000
APX,2019-06-14,25.82,0.005689,0.033277,1.100,90.581,45.190,73.647,1.245,18.441,-3.447
MFG,2019-06-14,47.60,0.005717,0.016877,1.099,90.481,9.319,67.435,1.083,59.731,3.718
NAN,2019-06-14,4.93,0.005349,0.025646,1.096,90.381,29.960,77.455,0.793,95.161,9.939
BRG,2019-06-14,16.97,0.005061,0.023582,0.992,88.978,25.451,55.411,0.933,71.183,1.414
IEL,2019-06-14,18.03,0.005005,0.026398,0.960,88.577,31.764,71.042,0.964,53.065,7.321
DHG,2019-06-14,3.34,0.004529,0.033834,0.883,86.072,45.992,84.168,0.842,67.097,2.994
CHC,2019-06-14,11.52,0.004294,0.011989,0.826,85.271,2.655,83.467,0.743,76.398,5.035


In [14]:
# high last week return
df.sort_values(by='week_return', ascending=False).head(20).sort_values(by='return', ascending=False)

,last_date,start price,return_mean,return_sigma,return,return_rank,risk_rank,volume_rank,last_return,return_increase_rank,week_return
code,,,,,,,,,,,
DUB,2019-06-14,1.420,0.014615,0.059723,2.810,98.798,76.253,70.741,2.318,98.495,21.127
5GN,2019-06-14,1.420,0.010393,0.046456,1.987,96.894,59.619,61.323,1.720,93.441,17.606
ALK,2019-06-14,0.415,0.009412,0.049031,1.814,96.293,64.429,80.160,1.377,97.957,18.072
M7T,2019-06-14,0.450,0.009076,0.041900,1.773,96.092,53.106,60.120,1.278,98.602,20.000
FMS,2019-06-14,0.052,0.006501,0.068578,1.288,92.485,88.878,46.593,0.434,99.677,28.846
SVM,2019-06-14,0.120,0.006250,0.040174,1.277,92.285,53.908,64.128,0.812,98.280,17.500
SO4,2019-06-14,0.800,0.006615,0.030287,1.250,92.084,35.872,33.267,0.838,97.527,20.000
BOT,2019-06-14,0.130,0.006368,0.042370,1.240,91.683,56.613,76.353,0.814,97.634,23.077
CLA,2019-06-14,0.064,0.007115,0.101932,1.109,90.681,99.299,98.196,0.357,99.462,32.813


## Thisk week recommendations

In [15]:
# recommendations
selection = [
    'WKT',  # high return & high risk
    'MGR',  # high price & low risk
    'MFG',  # high price & low sigma & low risk & asx 200
    'CHC',  # low sigma & low sigma & low risk & asx 200
    'TLS',  # high volume & low risk & asx 20
    'TCL',  # high volume & low risk & asx 20
    'AZJ',  # high volume & low risk
    'M7T',  # medium risk & best last week performance
]
df[df.index.isin(selection)].sort_values(by='return', ascending=False).head(10)

,last_date,start price,return_mean,return_sigma,return,return_rank,risk_rank,volume_rank,last_return,return_increase_rank,week_return
code,,,,,,,,,,,
WKT,2019-06-14,0.35,0.014051,0.051570,2.798,98.697,61.924,73.347,2.930,20.269,-5.714
M7T,2019-06-14,0.45,0.009076,0.041900,1.773,96.092,53.106,60.120,1.278,98.602,20.000
MFG,2019-06-14,47.60,0.005717,0.016877,1.099,90.481,9.319,67.435,1.083,59.731,3.718
CHC,2019-06-14,11.52,0.004294,0.011989,0.826,85.271,2.655,83.467,0.743,76.398,5.035
MGR,2019-06-14,3.21,0.003611,0.011091,0.697,82.114,2.104,99.599,0.621,74.946,2.492
TLS,2019-06-14,3.87,0.003052,0.012511,0.621,79.559,5.010,100.000,0.521,78.925,3.876
AZJ,2019-06-14,5.34,0.002219,0.010764,0.430,69.539,3.507,98.096,0.400,63.333,2.434
TCL,2019-06-14,14.51,0.002049,0.008477,0.406,67.635,1.503,95.491,0.380,62.419,1.723


## Last week review

In [16]:
selection = [
    'FEX',  # high return & high risk
    'ANO',  # high return & high price & low sigma
    'AD8',  # high price & low sigma & low risk
    'MFG',  # high price & low sigma & low risk
    'CHC',  # low sigma & low risk & asx 200
    'JBH',  # low sigma & low risk & asx 200
    'CV1',  # best recent performance
    'FMG'
]
last_week_df = df[df.index.isin(selection)]
last_week_df.sort_values(by='return', ascending=False).head(10)

,last_date,start price,return_mean,return_sigma,return,return_rank,risk_rank,volume_rank,last_return,return_increase_rank,week_return
code,,,,,,,,,,,
FEX,2019-06-14,0.110,0.019293,0.089630,3.848,99.699,94.790,98.898,4.226,4.785,-4.545
ANO,2019-06-14,5.920,0.018382,0.068696,3.653,99.399,80.561,28.257,3.785,20.269,-9.459
CV1,2019-06-14,0.145,0.010208,0.054010,1.954,96.693,70.240,53.006,2.377,3.441,0.000
FMG,2019-06-14,8.800,0.007762,0.029624,1.586,94.990,33.166,99.900,1.370,90.645,10.795
AD8,2019-06-14,7.940,0.008233,0.026914,1.573,94.840,27.255,44.890,1.603,45.806,2.267
MFG,2019-06-14,47.600,0.005717,0.016877,1.099,90.481,9.319,67.435,1.083,59.731,3.718
CHC,2019-06-14,11.520,0.004294,0.011989,0.826,85.271,2.655,83.467,0.743,76.398,5.035
JBH,2019-06-14,26.580,0.002600,0.014466,0.506,74.048,9.920,72.144,0.655,17.688,-4.289


In [17]:
week_return = round(sum(last_week_df['week_return'])/len(selection), 3)
win_rate = round(len(last_week_df[last_week_df['week_return'] >= 0]) / len(selection), 3)
print('Last week average return:', week_return, ' , win rate:', win_rate)

Last week average return: 0.44  , win rate: 0.625


## Signle performance review

In [18]:
def inspect(code):
    print(f'http://h.luotao.net/data/{this_week}/pic/{code}_line.png')
    print(f'http://h.luotao.net/data/{this_week}/pic/{code}.png')
    
    return df[df.index==code]

In [19]:
inspect('TNY')

http://h.luotao.net/data/20190614/pic/TNY_line.png
http://h.luotao.net/data/20190614/pic/TNY.png


,last_date,start price,return_mean,return_sigma,return,return_rank,risk_rank,volume_rank,last_return,return_increase_rank,week_return
code,,,,,,,,,,,
TNY,2019-06-14,1.19,0.018666,0.109975,3.639,99.299,99.198,31.363,NaN,NaN,NaN


In [20]:
df['rank'] = df['return_rank'] + (100 - df['risk_rank'])
top_df = df.sort_values(by='rank', ascending=False).head(20)
week_return = round(sum(top_df['week_return'].dropna())/len(top_df), 3)
print('Top rank last week average return:', week_return)
top_df

Top rank last week average return: 3.504


,last_date,start price,return_mean,return_sigma,return,return_rank,risk_rank,volume_rank,last_return,return_increase_rank,week_return,rank
code,,,,,,,,,,,,
CHC,2019-06-14,11.520,0.004294,0.011989,0.826,85.271,2.655,83.467,0.743,76.398,5.035,182.616
MFG,2019-06-14,47.600,0.005717,0.016877,1.099,90.481,9.319,67.435,1.083,59.731,3.718,181.162
MGR,2019-06-14,3.210,0.003611,0.011091,0.697,82.114,2.104,99.599,0.621,74.946,2.492,180.010
TLS,2019-06-14,3.870,0.003052,0.012511,0.621,79.559,5.010,100.000,0.521,78.925,3.876,174.549
DDR,2019-06-14,5.620,0.007496,0.024108,1.446,94.088,21.343,38.778,1.245,89.677,9.964,172.745
ASX,2019-06-14,81.000,0.002568,0.009643,0.496,73.597,1.904,57.214,0.495,54.731,2.111,171.693
MVF,2019-06-14,1.340,0.004062,0.017080,0.766,83.367,12.224,57.916,0.713,70.753,0.746,171.143
DTL,2019-06-14,2.140,0.003196,0.014170,0.613,79.158,9.018,49.699,0.644,45.538,8.879,170.140
CKF,2019-06-14,8.350,0.003189,0.015123,0.626,79.659,9.619,47.796,0.590,65.376,2.994,170.040


In [23]:
print(list(top_df.index))

['CHC', 'MFG', 'MGR', 'TLS', 'DDR', 'ASX', 'MVF', 'DTL', 'CKF', 'ASB', 'EML', 'GMG', 'APE', 'ABP', 'AD8', 'CNI', 'APA', 'RIO', 'TCL', 'MND']
